In [1]:
import torch
import numpy as np
import random
from IPython import display
from matplotlib import pyplot as plt
from torch import nn
import torch.utils.data as Data
import torch.optim as optim
from torch.nn import init
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  #防止jupyter崩溃

In [2]:
#自定义数据
num_inputs = 500
num_examples = 10000
true_w = torch.ones(500,1)*0.0056
true_b = 0.028
x_features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
y_labels = torch.mm(x_features,true_w) + true_b
y_labels += torch.tensor(np.random.normal(0, 0.01, size=y_labels.size()), dtype=torch.float)

In [3]:
#实现FlattenLayer层
class FlattenLayer(torch.nn.Module):
    def __init__(self):
         super(FlattenLayer, self).__init__()
    def forward(self, x):
        return x.view(x.shape[0],-1)

In [4]:
batch_size = 50
#模型定义和参数初始化
num_hiddens,num_outputs = 256,1
net = nn.Sequential(
        FlattenLayer(),
        nn.Linear(num_inputs,num_hiddens),
        nn.ReLU(),
        nn.Linear(num_hiddens,num_outputs)
        )

for params in net.parameters():
    init.normal_(params,mean=0,std=0.01)


In [5]:
def get_kfold_data(k,i,X,y):
    """
    k:一个K折
    i:第几折
    X:
    """
    fold_size=X.shape[0]//k  # X是数据集，将所有数据集分为K份
    val_start=i*fold_size  #
    
    if i!=k-1: # 不是最后一折
        val_end=(i+1)*fold_size   
        
        ## 验证集
        X_valid,y_valid =X[val_start:val_end],y[val_start:val_end]
        ## 训练集
        X_train=torch.cat((X[0:val_start],X[val_end:]),dim=0)
        y_train=torch.cat((y[0:val_start],y[val_end:]),dim=0)
    else:
        X_valid,y_valid=X[val_start:],y[val_start:]  # 若不能整除，将多的样本放在最后一折里
        X_train=X[0:val_start]
        y_train=y[0:val_start]
    return X_train,y_train,X_valid,y_valid

In [6]:
#定义模型训练函数
def train(net,train_iter,test_iter,loss,num_epochs,batch_size,trainfeatures,trainlabels,testfeatures,testlabels,params=None,lr=None,optimizer=None):
    train_ls = []
    test_ls = []
    for epoch in range(num_epochs): # 训练模型一共需要num_epochs个迭代周期
        train_l_sum,n = 0.0,0
        # 在每一个迭代周期中，会使用训练数据集中所有样本一次
        for X, y in train_iter: # x和y分别是小批量样本的特征和标签
            y_hat = net(X)
            l = loss(y_hat, y.view(-1,1)) # l是有关小批量X和y的损失
            #梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward() # 小批量的损失对模型参数求梯度
            if optimizer is None:
                SGD(params,lr)
            else:
                optimizer.step()
        train_labels = trainlabels
        test_labels = testlabels
        train_ls.append(loss(net(trainfeatures),train_labels).item())
        test_ls.append(loss(net(testfeatures),test_labels).item())
    print('epoch %d, train_loss %.6f,test_loss %f'%(epoch+1, train_ls[epoch],test_ls[epoch]))
    return train_ls[epoch],test_ls[epoch]

In [7]:
def k_fold(k,X_train,y_train):
    train_loss_sum,valid_loss_sum= 0,0
    for i in range(k):
        print("第",i+1,"折验证结果")
        X_train,y_train,X_valid,y_valid=get_kfold_data(k,i,X_train,y_train)  # 获取K折的训练集和验证集
        
        train_dataset = Data.TensorDataset(X_train, y_train) 
        train_iter = Data.DataLoader(
                dataset=train_dataset, # torch TensorDataset format
                batch_size=batch_size, # mini batch size
                shuffle=True, # 是否打乱数据 (训练集一般需要进行打乱)
                num_workers=0, # 多线程来读数据， 注意在Windows下需要设置为0
                    )
        
        test_dataset = Data.TensorDataset(X_valid, y_valid)
        test_iter = Data.DataLoader(
                dataset=test_dataset, # torch TensorDataset format
                batch_size=batch_size, # mini batch size
                shuffle=True, # 是否打乱数据 (训练集一般需要进行打乱)
                num_workers=0, # 多线程来读数据， 注意在Windows下需要设置为0
                    )
        lr = 0.01
        #损失函数和优化器
        loss = torch.nn.MSELoss()
        optimizer = torch.optim.SGD(net.parameters(),lr)
        for params in net.parameters():
            init.normal_(params,mean=0,std=0.01) #每轮训练都重新初始化参数
        train_loss,test_loss = train(net,train_iter,test_iter,loss,num_epochs,batch_size,X_train,y_train,X_valid,y_valid,net.parameters,lr,optimizer)
        train_loss_sum+=train_loss
        valid_loss_sum+=test_loss
    print("\n最终k折交叉验证结果：")
    print("average train loss:{:.4f},average valis loss:{:.4f}".format(train_loss_sum/k,valid_loss_sum/k))
    return

In [8]:
#训练次数和学习率
num_epochs = 20
k_fold(10,x_features,y_labels)

第 1 折验证结果
epoch 20, train_loss 0.000743,test_loss 0.001140
第 2 折验证结果
epoch 20, train_loss 0.000795,test_loss 0.001315
第 3 折验证结果
epoch 20, train_loss 0.000870,test_loss 0.001377
第 4 折验证结果
epoch 20, train_loss 0.000914,test_loss 0.001414
第 5 折验证结果
epoch 20, train_loss 0.001056,test_loss 0.001804
第 6 折验证结果
epoch 20, train_loss 0.001288,test_loss 0.002159
第 7 折验证结果
epoch 20, train_loss 0.001742,test_loss 0.003139
第 8 折验证结果
epoch 20, train_loss 0.002139,test_loss 0.003518
第 9 折验证结果
epoch 20, train_loss 0.003209,test_loss 0.004710
第 10 折验证结果
epoch 20, train_loss 0.003853,test_loss 0.005458

最终k折交叉验证结果：
average train loss:0.0017,average valis loss:0.0026
